# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [23]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [24]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [25]:
symbol = 'AAPL'
base_url = 'https://sandbox.iexapis.com/stable'
api_url = f'{base_url}/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 70188225, 'calculationPrice': 'close', 'change': -0.86, 'changePercent': -0.00577, 'close': 153.7, 'closeSource': 'iafolicf', 'closeTime': 1669082770718, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 153.521, 'delayedPriceTime': 1641309225073, 'extendedChange': 0.04, 'extendedChangePercent': 0.00028, 'extendedPrice': 153.45, 'extendedPriceTime': 1704893774665, 'high': 154.66, 'highSource': 'yd delmaiicte epen1r u5', 'highTime': 1675789122513, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 149.302, 'iexCloseTime': 1648126824105, 'iexLastUpdated': 1674862364280, 'iexMarketPercent': 0.01103092080364732, 'iexOpen': 150.53, 'iexOpenTime': 1689135499240, 'iexRealtimePrice': 152.94, 'iexRealtimeSize': 10, 'iexVolume': 543692, 'lastTradeTime': 1681564075212, 'latestPrice': 151.48, 'latestSource': 'Close', 'latestTime': 'August 26, 2021', 'latestUpdate': 1645330064394, 'latestVolume': 48781296, 'low': 149.18, 'lowSource': 'l

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [26]:
price = data['latestPrice']
pe_ratio = data['peRatio']
price, pe_ratio

(151.48, 29.09)

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [27]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i: i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [28]:
def quantitative_value_data():
    dataframe = pd.DataFrame(columns = my_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f'{base_url}/stock/market/batch?symbols={symbols}&types=quote&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()
        # print(data['AAPL'])
        # print(data['AAPL']['stats'])

        for symbol in symbols.split(','):
            price = data[symbol]['quote']['latestPrice']
            pe_ratio = data[symbol]['quote']['peRatio']
            buy_count = np.nan

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        pe_ratio,
                        buy_count
                    ],
                    index = my_columns
                ),
                ignore_index = True
            )

    return dataframe.fillna(0)

final_dataframe = quantitative_value_data()
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,182.17,59.33,0.0
1,AAL,20.30,-2.12,0.0
2,AAP,211.55,22.66,0.0
3,AAPL,147.55,29.73,0.0
4,ABBV,121.90,34.01,0.0
...,...,...,...,...
500,YUM,131.50,31.14,0.0
501,ZBH,152.73,35.68,0.0
502,ZBRA,583.20,42.35,0.0
503,ZION,56.96,5.59,0.0


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [29]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis = 1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,78.75,4.09,0.0
1,MPC,58.81,5.14,0.0
2,ZION,56.96,5.59,0.0
3,GM,49.15,5.76,0.0
4,PRU,107.07,5.90,0.0
5,BIO,803.42,6.16,0.0
6,CINF,125.01,7.02,0.0
7,AFL,58.80,7.09,0.0
8,COF,176.15,7.13,0.0
9,CE,164.66,7.47,0.0


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [30]:
def portfolio_input():
    global portfolio_size
    portfolio_size = 2500000 # input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [31]:
portfolio_input()
portfolio_size

2500000

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [32]:
position_size = float(portfolio_size) / len(final_dataframe.index)

for row in final_dataframe.index:
    price = final_dataframe.loc[row, 'Price']
    buy_count =  math.floor(position_size / price)
    final_dataframe.loc[row, 'Number of Shares to Buy'] = buy_count

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,EBAY,78.75,4.09,634.0
1,MPC,58.81,5.14,850.0
2,ZION,56.96,5.59,877.0
3,GM,49.15,5.76,1017.0
4,PRU,107.07,5.90,466.0
5,BIO,803.42,6.16,62.0
6,CINF,125.01,7.02,399.0
7,AFL,58.80,7.09,850.0
8,COF,176.15,7.13,283.0
9,CE,164.66,7.47,303.0


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [33]:
symbol = 'AAPL'
api_url = f'{base_url}/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'AAPL': {'advanced-stats': {'beta': 1.545038052010898, 'totalCash': 62080392021, 'currentDebt': 109539563704, 'revenue': 356992254993, 'grossProfit': 143974139412, 'totalRevenue': 363631107976, 'EBITDA': 111768819449, 'revenuePerShare': 22, 'revenuePerEmployee': 2425622.41, 'debtToEquity': 5.14889247433028, 'profitMargin': 0.2549612030554276, 'enterpriseValue': 2532882876001, 'enterpriseValueToRevenue': 7.53, 'priceToSales': 7.34, 'priceToBook': 39.92, 'forwardPERatio': 0.00238505301805778, 'pegRatio': 0.5395786004495106, 'peHigh': 13.919094786909895, 'peLow': 10.606554743584395, 'week52highDate': '2021-08-10', 'week52lowDate': '2020-09-06', 'putCallRatio': 0.4517998947590636, 'companyName': 'Apple Inc', 'marketcap': 2462371731789, 'week52high': 155.84, 'week52low': 102.59, 'week52highSplitAdjustOnly': 154.32, 'week52highDateSplitAdjustOnly': '2021-08-08', 'week52lowSplitAdjustOnly': 104.1, 'week52lowDateSplitAdjustOnly': '2020-09-12', 'week52change': 0.18608474654126128, 'sharesOutst

In [34]:
# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']


# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = ev / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = ev / gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [35]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score',
]

pd.DataFrame(columns = rv_columns)

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [36]:
na = 'N/A'
def robust_value_data():
    dataframe = pd.DataFrame(columns = rv_columns)

    for symbols in symbol_strings:
        # print(symbols)
        api_url = f'{base_url}/stock/market/batch?symbols={symbols}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
        data = requests.get(api_url).json()

        for symbol in symbols.split(','):
            price = data[symbol]['quote']['latestPrice']
            pe_ratio = data[symbol]['quote']['peRatio']
            buy_count = na
            
            pb_ratio = data[symbol]['advanced-stats']['priceToBook']
            ps_ratio = data[symbol]['advanced-stats']['priceToSales']

            ev = data[symbol]['advanced-stats']['enterpriseValue']
            ebitda = data[symbol]['advanced-stats']['EBITDA']
            try:
                ev_to_ebitda = ev / ebitda
            except TypeError:
                ev_to_ebitda = np.nan

            gross_profit = data[symbol]['advanced-stats']['grossProfit']
            try:
                ev_to_gross_profit = ev / gross_profit
            except TypeError:
                ev_to_gross_profit = np.nan

            dataframe = dataframe.append(
                pd.Series(
                    [
                        symbol,
                        price,
                        buy_count,
                        pe_ratio,
                        # 'PE Percentile'
                        na,
                        pb_ratio,
                        # 'PB Percentile',
                        na,
                        ps_ratio,
                        # 'PS Percentile',
                        na,
                        ev_to_ebitda,
                        # 'EV/EBITDA Percentile',
                        na,
                        ev_to_gross_profit,
                        # 'EV/GP Percentile',
                        na,
                        # 'RV Score',
                        na,
                    ],
                    index = rv_columns
                ),
                ignore_index = True
            )

    return dataframe #.fillna(0)

rv_dataframe = robust_value_data()
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,178.38,N/A,60.79,N/A,10.95,N/A,9.06,N/A,38.244260,N/A,17.539112,N/A,N/A
1,AAL,20.60,N/A,-2.08,N/A,-1.7,N/A,0.7242,N/A,-4.673915,N/A,1.909734,N/A,N/A
2,AAP,210.88,N/A,22.52,N/A,4.01,N/A,1.31,N/A,12.509211,N/A,2.990748,N/A,N/A
3,AAPL,149.20,N/A,29.56,N/A,38.69,N/A,7.2,N/A,23.061584,N/A,17.935708,N/A,N/A
4,ABBV,125.40,N/A,33.64,N/A,17.55,N/A,4.01,N/A,11.584060,N/A,8.237436,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,133.80,N/A,31.19,N/A,-5.03,N/A,6.6,N/A,23.061878,N/A,10.739242,N/A,N/A
501,ZBH,152.62,N/A,35.5,N/A,2.5,N/A,4.16,N/A,20.042149,N/A,6.685198,N/A,N/A
502,ZBRA,581.82,N/A,41.45,N/A,12.39,N/A,6.07,N/A,29.806176,N/A,12.965752,N/A,N/A
503,ZION,56.83,N/A,5.71,N/A,1.25,N/A,2.65,N/A,4.656089,N/A,2.443096,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [37]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,183.300,N/A,59.54,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,281.700,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,297.240,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,10.09,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,27.910,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.130,N/A,14.37,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.630,N/A,33.28,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,35.000,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2884.310,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.182,N/A,30.99,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [42]:
columns = ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']
for column in columns:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [43]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [56]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric]) / 100

rv_dataframe

/opt/homebrew/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.93,1856,8.12,0.46,0.5093,0.14,0.4264,0.12,3.662946,0.16,0.413920,0.04,4.752475
1,PRU,110.47,452,6.00,0.28,0.6790,0.18,0.6600,0.22,4.806905,0.26,0.653357,0.06,5.188119
2,AAL,20.60,2427,-2.08,0.22,-1.7000,0.12,0.7242,0.28,-4.673915,0.04,1.909734,0.4,5.782178
3,GM,49.94,1001,5.62,0.24,1.4000,0.67,0.5360,0.16,2.194965,0.12,1.994396,0.48,7.722772
4,MCK,207.12,241,-7.00,0.16,-59.8900,0.04,0.1296,0.02,8.318295,0.84,2.879340,0.78,8.039604
5,L,55.70,897,10.15,0.62,0.8326,0.28,1.0038,0.42,5.584448,0.38,0.953791,0.14,8.118812
6,BA,222.50,224,-14.84,0.12,-7.8000,0.08,2.2000,0.84,-31.489970,0.02,-214.869926,0.02,8.455446
7,LNC,71.39,700,9.54,0.58,0.6262,0.16,0.6881,0.26,7.854883,0.8,0.680913,0.08,8.594059
8,MET,63.33,789,12.66,0.82,0.8079,0.24,0.8070,0.34,5.963289,0.48,0.765363,0.1,8.712871
9,RE,268.84,185,8.08,0.44,1.0400,0.42,0.9680,0.38,7.330186,0.7,0.994626,0.18,8.950495


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [57]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)

rv_dataframe

/opt/homebrew/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.93,1856,8.12,0.46,0.5093,0.14,0.4264,0.12,3.662946,0.16,0.413920,0.04,0.184
1,PRU,110.47,452,6.00,0.28,0.6790,0.18,0.6600,0.22,4.806905,0.26,0.653357,0.06,0.2
2,AAL,20.60,2427,-2.08,0.22,-1.7000,0.12,0.7242,0.28,-4.673915,0.04,1.909734,0.4,0.212
3,GM,49.94,1001,5.62,0.24,1.4000,0.67,0.5360,0.16,2.194965,0.12,1.994396,0.48,0.334
4,MCK,207.12,241,-7.00,0.16,-59.8900,0.04,0.1296,0.02,8.318295,0.84,2.879340,0.78,0.368
5,L,55.70,897,10.15,0.62,0.8326,0.28,1.0038,0.42,5.584448,0.38,0.953791,0.14,0.368
6,BA,222.50,224,-14.84,0.12,-7.8000,0.08,2.2000,0.84,-31.489970,0.02,-214.869926,0.02,0.216
7,LNC,71.39,700,9.54,0.58,0.6262,0.16,0.6881,0.26,7.854883,0.8,0.680913,0.08,0.376
8,MET,63.33,789,12.66,0.82,0.8079,0.24,0.8070,0.34,5.963289,0.48,0.765363,0.1,0.396
9,RE,268.84,185,8.08,0.44,1.0400,0.42,0.9680,0.38,7.330186,0.7,0.994626,0.18,0.424


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [58]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

/opt/homebrew/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.93,1856,8.12,0.46,0.5093,0.14,0.4264,0.12,3.662946,0.16,0.413920,0.04,0.184
1,PRU,110.47,452,6.00,0.28,0.6790,0.18,0.6600,0.22,4.806905,0.26,0.653357,0.06,0.2
2,AAL,20.60,2427,-2.08,0.22,-1.7000,0.12,0.7242,0.28,-4.673915,0.04,1.909734,0.4,0.212
3,BA,222.50,224,-14.84,0.12,-7.8000,0.08,2.2000,0.84,-31.489970,0.02,-214.869926,0.02,0.216
4,GM,49.94,1001,5.62,0.24,1.4000,0.67,0.5360,0.16,2.194965,0.12,1.994396,0.48,0.334
5,MCK,207.12,241,-7.00,0.16,-59.8900,0.04,0.1296,0.02,8.318295,0.84,2.879340,0.78,0.368
6,L,55.70,897,10.15,0.62,0.8326,0.28,1.0038,0.42,5.584448,0.38,0.953791,0.14,0.368
7,LNC,71.39,700,9.54,0.58,0.6262,0.16,0.6881,0.26,7.854883,0.8,0.680913,0.08,0.376
8,MET,63.33,789,12.66,0.82,0.8079,0.24,0.8070,0.34,5.963289,0.48,0.765363,0.1,0.396
9,AIG,55.18,906,11.64,0.76,0.7225,0.2,1.0700,0.47,5.738486,0.42,1.062264,0.2,0.41


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [59]:
portfolio_input()
portfolio_size

2500000

In [60]:
position_size = float(portfolio_size) / len(rv_dataframe.index)

for row in rv_dataframe.index:
    price = rv_dataframe.loc[row, 'Price']
    buy_count = math.floor(position_size / price)
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = buy_count

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.93,1856,8.12,0.46,0.5093,0.14,0.4264,0.12,3.662946,0.16,0.413920,0.04,0.184
1,PRU,110.47,452,6.00,0.28,0.6790,0.18,0.6600,0.22,4.806905,0.26,0.653357,0.06,0.2
2,AAL,20.60,2427,-2.08,0.22,-1.7000,0.12,0.7242,0.28,-4.673915,0.04,1.909734,0.4,0.212
3,BA,222.50,224,-14.84,0.12,-7.8000,0.08,2.2000,0.84,-31.489970,0.02,-214.869926,0.02,0.216
4,GM,49.94,1001,5.62,0.24,1.4000,0.67,0.5360,0.16,2.194965,0.12,1.994396,0.48,0.334
5,MCK,207.12,241,-7.00,0.16,-59.8900,0.04,0.1296,0.02,8.318295,0.84,2.879340,0.78,0.368
6,L,55.70,897,10.15,0.62,0.8326,0.28,1.0038,0.42,5.584448,0.38,0.953791,0.14,0.368
7,LNC,71.39,700,9.54,0.58,0.6262,0.16,0.6881,0.26,7.854883,0.8,0.680913,0.08,0.376
8,MET,63.33,789,12.66,0.82,0.8079,0.24,0.8070,0.34,5.963289,0.48,0.765363,0.1,0.396
9,AIG,55.18,906,11.64,0.76,0.7225,0.2,1.0700,0.47,5.738486,0.42,1.062264,0.2,0.41


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [61]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [62]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format({
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

dollar_template = writer.book.add_format({
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

integer_template = writer.book.add_format({
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

float_template = writer.book.add_format({
    'num_format': '0.0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

percent_template = writer.book.add_format({
    'num_format': '0.0%',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
})

In [65]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', float_template],
}

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

NameError: name 'percent_template' is not defined

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [ ]:
writer.save()